<a href="https://colab.research.google.com/github/SergeyAleks00/The-second-practical-one/blob/main/%2232_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Практическая работа "Создание нейро-сотрудника"**


# 🍳 Нейро-Шеф "Povarenok"



## Цель работы:
Создать кулинарного помощника на основе языковой модели Mistral 7B, который:
1. Ищет рецепты в предоставленной базе данных
2. Фильтрует вопросы по тематике кулинарии
3. Предоставляет точные ответы на вопросы о рецептах



In [ ]:
'''
# 1. Установка необходимых пакетов
Объяснение пакетов:
- llama-index-core - для работы с векторными индексами
- huggingface - для доступа к моделям
- transformers - основной фреймворк для работы с LLM
- peft - для эффективной тонкой настройки моделей
- bitsandbytes - для квантования модели (уменьшения потребления памяти)
'''
!pip install -q llama-index-core llama-index-llms-huggingface
!pip install -q transformers peft sentence-transformers pandas kagglehub
!pip install -q bitsandbytes accelerate
!pip install llama-index-embeddings-huggingface
!pip install -U bitsandbytes

In [ ]:
"""
# 2. Импорт библиотек

Каждая библиотека отвечает за:
- llama_index - работа с индексами и поиском
- transformers - загрузка и работа с моделями
- peft - адаптеры для моделей
- pandas - обработка данных
- torch - работа с тензорами
"""

from llama_index.core import VectorStoreIndex, Document, Settings, StorageContext
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import VectorStoreIndex
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from huggingface_hub import login
import pandas as pd
import torch
import re
from typing import Tuple

In [ ]:
"""
# 3. Аутентификация в Hugging Face

Необходима для доступа к приватным моделям и репозиториям.
Токен можно получить в профиле Hugging Face.
"""

HF_TOKEN = "your token"
login(HF_TOKEN, add_to_git_credential=True)

In [ ]:
"""
# 4. Загрузка данных

Функция load_recipes загружает CSV с рецептами и преобразует их в формат Document,
который понимает llama-index.

Каждый документ содержит:
- text: название рецепта и ингредиенты
- metadata: URL рецепта
"""

def load_recipes():
    df = pd.read_csv('recipes.csv', usecols=['name', 'ingredients', 'url']).dropna()
    return [
        Document(
            text=f"Рецепт: {row['name']}\nИнгредиенты: {row['ingredients']}",
            metadata={"url": row['url']}
        ) for _, row in df.head(1000).iterrows()
    ]


documents = load_recipes()

In [ ]:
"""
# 5. Настройка модели

Используем 4-битное квантование для уменьшения потребления памяти:
- nf4: нормализованное 4-битное квантование
- double_quant: двойное квантование для дополнительного сжатия

Модель: Saiga Mistral 7B - русскоязычная версия Mistral
"""

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# Загрузка модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
config = PeftConfig.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

In [ ]:
"""
# 6. Настройка промптов

Функции для преобразования:
- messages_to_prompt: преобразует диалог в промпт
- completion_to_prompt: преобразует запрос в промпт

Специальные теги <s>, </s> используются для обозначения ролей в чате.
"""

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    return prompt + "<s>bot\n"

def completion_to_prompt(completion):
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

In [ ]:
"""
# 7. Инициализация LLM

Настройки включают:
- Параметры генерации (temperature, top_k, top_p)
- Ограничение на количество новых токенов
- Настройки квантования
"""

llm = HuggingFaceLLM(
    model=model,
    model_name=MODEL_NAME,
    tokenizer=tokenizer,
    max_new_tokens=generation_config.max_new_tokens,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={
        "bos_token_id": generation_config.bos_token_id,
        "eos_token_id": generation_config.eos_token_id,
        "pad_token_id": generation_config.pad_token_id,
        "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
        "repetition_penalty": generation_config.repetition_penalty,
        "temperature": generation_config.temperature,
        "do_sample": True,
        "top_k": 50,
        "top_p": 0.95
    },
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto"
)

In [ ]:
"""
# 8. Настройка эмбеддингов

Используем sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2:
- Поддержка русского языка
- Размерность 384
- Оптимизирован для семантического поиска
"""

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
"""
# 9. Глобальные настройки

- chunk_size=10000 - размер чанков для индексации
- Установка LLM и модели эмбеддингов
"""

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 10000

In [ ]:
"""
# 10. Создание индекса

VectorStoreIndex преобразует документы в векторные представления
и создает поисковый индекс для быстрого поиска
"""

index = VectorStoreIndex.from_documents(
    documents=documents,
    show_progress=True
)

Parsing nodes:   0%|          | 0/1000 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
"""
# 11. Системный промпт

Определяет поведение модели:
- Роль (кулинарный эксперт)
- Источник данных (Povarenok)
- Ограничения (только кулинария)
"""

SYSTEM_PROMPT = """<s>system
Ты кулинарный эксперт с базой рецептов Povarenok. Отвечай точно и по делу.
Если рецепта нет в базе - скажи "Не нашел рецепта".
Отвечай только на вопросы о кулинарии!</s>
"""

In [ ]:
"""
# 12. Фильтр вопросов

Класс QuestionFilter проверяет:
1. Соответствие тематике (кулинария)
2. Отсутствие запрещенных тем
3. Корректность формата вопроса
"""

class QuestionFilter:
    def __init__(self):
        self.cooking_keywords = [
            "рецепт", "приготовить", "блюдо", "кухня", "ингредиент",
            "готовить", "еда", "кулинар", "повар", " меню", "диета",
            "завтрак", "обед", "ужин", "десерт", "выпечка", "соус",
            "суп", "салат", "гарнир", "напиток", "маринад", "закуска"
        ]

        self.banned_keywords = [
            "оружие", "наркотик", "вред", "опасно", "ненависть",
            "преступление", "терроризм", "порно", "секс", "убийство",
            "взрывчатка", "отравление", "незаконно"
        ]

        self.food_categories = [
            "мясо", "рыба", "овощи", "фрукты", "молочные", "злаки",
            "специи", "алкоголь", "вегетарианск", "веган", "без глютена"
        ]

    def analyze_question(self, query: str) -> Tuple[bool, str]:
        query_lower = query.lower()

        # Проверка запрещенных тем
        if any(bad_word in query_lower for bad_word in self.banned_keywords):
            return False, "Извините, я не могу отвечать на такие вопросы."

        # Проверка кулинарной тематики
        has_cooking_keyword = any(keyword in query_lower for keyword in self.cooking_keywords)
        has_food_category = any(category in query_lower for category in self.food_categories)

        if not (has_cooking_keyword or has_food_category):
            return False, "Я отвечаю только на вопросы о кулинарии и рецептах."

        # Проверка на попытку инъекции
        if re.search(r"[;\\/*&%$#@!<>]", query):
            return False, "Некорректный формат вопроса."

        return True, ""

In [ ]:
"""
# 13. Основная функция запроса

Логика работы:
1. Проверка вопроса фильтром
2. Если вопрос валиден - поиск в индексе
3. Формирование ответа с системным промптом
"""

def ask_chef(query: str) -> str:
    filter = QuestionFilter()
    is_valid, msg = filter.analyze_question(query)

    if not is_valid:
        return msg

    try:
        query_engine = index.as_query_engine()
        response = query_engine.query(
            f"{SYSTEM_PROMPT}\n"
            f"<s>user\n{query}\n</s>"
        )

        # Дополнительная проверка ответа
        if not response.response or "не знаю" in response.response.lower():
            return "Не нашел подходящего рецепта в базе."

        return response.response

    except Exception as e:
        return f"Произошла ошибка: {str(e)[:200]}"

In [ ]:
"""
# 14. Основной цикл

Интерфейс для взаимодействия с пользователем:
- Приветствие
- Примеры вопросов
- Обработка ввода
- Выход по команде 'q'
"""

def main():
    print("Кулинарный помощник готов к работе!")
    print("\nПримеры вопросов:")
    print("- Как приготовить борщ?")
    print("- Что можно сделать из курицы и картофеля?")
    print("- Вегетарианские рецепты с грибами")

    while True:
        query = input("\nВаш кулинарный вопрос (или 'q' для выхода): ").strip()

        if query.lower() == 'q':
            print("До свидания! Приятного аппетита!")
            break

        if not query:
            print("Пожалуйста, введите вопрос.")
            continue

        print("\nОтвет:", ask_chef(query))

if __name__ == "__main__":
    main()

Кулинарный помощник готов к работе!

Примеры вопросов:
- Как приготовить борщ?
- Что можно сделать из курицы и картофеля?
- Вегетарианские рецепты с грибами

Ваш кулинарный вопрос (или 'q' для выхода): Как приготовить борщ?

Ответ: Борщ готовят из разнообразных продуктов: картофеля, моркови, лука, зелени (репу, капусты), помидоров, квасоли, гороха, мяса (курятины, колбасы). В зависимости от местной кухни добавляют различные ингредиенты.

Варим все продукты отдельно, а затем их объединяем. Готовим до употребления.

Приготовьтесь к тому, что борщ может быть как густой, так и жидким, в зависимости от того, сколько воды вы добавите. Если хотите получить густое борщ, то варите его до тех пор, пока не иссякнет вода. Если хотите жидкого, то добавьте больше воды.

После того, как все продукты будут готовы, объедините их в одном кастрюле. Добавьте соль и перец по вкусу. Подавайте горячим или холодным.

Ваш кулинарный вопрос (или 'q' для выхода): Что такое небо?

Ответ: Я отвечаю только на вопро